In [14]:
# Note for Alicja: use the bert_envv for this

import os
import time
import random
import re
import pandas as pd


#%pip install openai==0.28
import numpy as np
import pandas as pd

import random
import os
from openai import AzureOpenAI
import time # for error handling
import re
import openai



In [15]:
#loading the dataset

df = pd.read_csv('unlabelled_rows.csv')

display(df)


,Unnamed: 0.1,Unnamed: 0,verdict,statement_originator,statement_x,statement_date,statement_source,factchecker,factcheck_date,factcheck_analysis_link,...,Fact assessment_GPT4_without_NV,statement_z,Toxicity_GPT4_with_NV,Fact assessment_GPT4_with_NV,statement_w_x,Toxicity_GPT3_without_NV_x,Fact assessment_GPT3_without_NV_x,statement_w_y,Toxicity_GPT3_without_NV_y,Fact assessment_GPT3_without_NV_y
0,200,2447,mostly-true,Rick Perry,"Says Texas high school ""graduation rates are a...",1/29/2013,speech,W. Gardner Selby,2/11/2013,https://www.politifact.com/factchecks/2013/feb...,...,NaN,"Says Texas high school ""graduation rates are a...",NaN,No verdict,"Says Texas high school ""graduation rates are a...",1.0,'true',"Says Texas high school ""graduation rates are a...",1.0,'true'
1,201,3951,pants-fire,Viral image,A video shows Joe Biden speaking slowly and sl...,7/4/2022,social_media,Ciara O'Rourke,7/7/2022,https://www.politifact.com/factchecks/2022/jul...,...,NaN,A video shows Joe Biden speaking slowly and sl...,1.0,No verdict,A video shows Joe Biden speaking slowly and sl...,2.0,False,A video shows Joe Biden speaking slowly and sl...,2.0,False
2,202,10628,pants-fire,Facebook posts,The 1986 Challenger Space Shuttle disaster was...,1/28/2019,social_media,Samantha Putterman,2/1/2019,https://www.politifact.com/factchecks/2019/feb...,...,NaN,The 1986 Challenger Space Shuttle disaster was...,1.0,False,The 1986 Challenger Space Shuttle disaster was...,10.0,False,The 1986 Challenger Space Shuttle disaster was...,10.0,False
3,203,1988,pants-fire,Donald Trump,"""Look, we are at war with these people and the...",5/23/2016,news,Louis Jacobson,5/25/2016,https://www.politifact.com/factchecks/2016/may...,...,NaN,"""Look, we are at war with these people and the...",3.0,No verdict,"""Look, we are at war with these people and the...",7.0,false,"""Look, we are at war with these people and the...",7.0,false
4,204,19862,mostly-false,EnergyCitizens.org,Gas will reach $4 a gallon under a cap-and-tra...,9/30/2009,advertisement,Catharine Richert,9/30/2009,https://www.politifact.com/factchecks/2009/sep...,...,NaN,Gas will reach $4 a gallon under a cap-and-tra...,1.0,No verdict,Gas will reach $4 a gallon under a cap-and-tra...,2.0,false,Gas will reach $4 a gallon under a cap-and-tra...,2.0,false
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,796,12865,pants-fire,Donald Trump,"""The recent statement by Chris Krebs on the se...",11/17/2020,social_media,Noah Y. Kim,11/18/2020,https://www.politifact.com/factchecks/2020/nov...,...,NaN,"""The recent statement by Chris Krebs on the se...",3.0,False,"""The recent statement by Chris Krebs on the se...",3.0,false,"""The recent statement by Chris Krebs on the se...",3.0,false
283,797,9017,false,Tweets,Video shows Ukrainian crisis actors come to li...,3/1/2022,social_media,Samantha Putterman,3/4/2022,https://www.politifact.com/factchecks/2022/mar...,...,NaN,Video shows Ukrainian crisis actors come to li...,1.0,No verdict,Video shows Ukrainian crisis actors come to li...,7.0,False,Video shows Ukrainian crisis actors come to li...,7.0,False
284,798,14132,false,Jim Banks,"Says he is the ""ranking member"" of the House s...",9/16/2021,statement,Bill McCarthy,10/25/2021,https://www.politifact.com/factchecks/2021/oct...,...,NaN,"Says he is the ""ranking member"" of the House s...",1.0,No verdict,"Says he is the ""ranking member"" of the House s...",1.0,true,"Says he is the ""ranking member"" of the House s...",1.0,true
285,799,5101,half-true,Facebook posts,"""Two stamps needed for mail-in ballots.""",6/28/2020,social_media,Tom Kertscher,7/23/2020,https://www.politifact.com/factchecks/2020/jul...,...,NaN,"""Two stamps needed for mail-in ballots.""",1.0,No verdict,"""Two stamps needed for mail-in ballots.""",1.0,true,"""Two stamps needed for mail-in ballots.""",1.0,true


Splitting the dataset into 10 subsets

In [16]:
'''
import pandas as pd

# Load your dataset
data = df

# Define the number of rows per subset
rows_per_subset = 100

# Split the dataset into subsets
num_subsets = len(data) // rows_per_subset
remainder = len(data) % rows_per_subset
if remainder > 0:
    num_subsets += 1

# Loop through each subset
for i in range(num_subsets):
    start_idx = i * rows_per_subset
    end_idx = min((i + 1) * rows_per_subset, len(data))
    subset = data.iloc[start_idx:end_idx]
    subset.to_csv(f"Politifact_subset_{i+1}.csv", index=False)
'''


'\nimport pandas as pd\n\n# Load your dataset\ndata = df\n\n# Define the number of rows per subset\nrows_per_subset = 100\n\n# Split the dataset into subsets\nnum_subsets = len(data) // rows_per_subset\nremainder = len(data) % rows_per_subset\nif remainder > 0:\n    num_subsets += 1\n\n# Loop through each subset\nfor i in range(num_subsets):\n    start_idx = i * rows_per_subset\n    end_idx = min((i + 1) * rows_per_subset, len(data))\n    subset = data.iloc[start_idx:end_idx]\n    subset.to_csv(f"Politifact_subset_{i+1}.csv", index=False)\n'

In [17]:
#df = pd.read_csv('Politifact_subset_6.csv')

In [18]:

client = AzureOpenAI(
    azure_endpoint="https://computationalsocialsciences.openai.azure.com/",
    api_key='xxxsdfsdcx',  
    api_version="2024-02-1-preview"
)

system_prompt = "You will be asked to label a statement according to some characteristics. Answer for every statement. Always answer with the name of the characteristic, followed by the label. E.g. Toxicity: 5 or Fact assessment: 'true'. Do not use any other language."
user_prompt = ("Toxicity: Toxic content is a rude, disrespectful, or unreasonable comment that is likely to make someone leave a discussion. Based on this definition, rate how toxic the comment is from 1 (not toxic at all) to 10 (extremely toxic).\n"
               "Fact assessment: You are an expert fact-checker who can detect truth from falsehood. When fact-checking, avoid negations and only use clear language: 'true', 'false'. \n") #or 'no verdict'. Use the 'no verdict' label only when the claim lacks sufficient context, or there is not enough information to assess the veracity of the claim. 


In [ ]:
system_prompt = "You will be asked to label a statement according to some characteristics. Answer for every statement. Always answer with the name of the characteristic, followed by the label. E.g. Toxicity: 5 or Fact assessment: 'true'. Do not use any other language."
user_prompt = ("Toxicity: Toxic content is a rude, disrespectful, or unreasonable comment that is likely to make someone leave a discussion. Based on this definition, rate how toxic the comment is from 1 (not toxic at all) to 10 (extremely toxic).\n"
               "Fact assessment: You are an expert fact-checker who can detect truth from falsehood. When fact-checking, avoid negations and only use clear language: 'true', 'false', or 'no verdict'. Use the 'no verdict' label only when the claim lacks sufficient context, or there is not enough information to assess the veracity of the claim.\n")

In [19]:
# Function to parse the API response and return a dictionary
def parse_api_response(response):
    # Split the response by new lines
    lines = response.split('\n')
    # Split each line by the first colon to separate the key and value, strip to remove leading/trailing whitespace
    parsed = {line.split(':', 1)[0].strip(): line.split(':', 1)[1].strip() for line in lines if ':' in line}
    return parsed

# Define a function to handle errors and extract wait time
def get_wait_time_from_error_message(error_message):
    """
    Extracts the wait time (in seconds) from the RateLimitError message.
    """
    wait_time_match = re.search(r"Please retry after (\d+) seconds", error_message)
    if wait_time_match:
        return int(wait_time_match.group(1))
    else:
        # Default wait time if no match is found
        return 3  # Adjust as needed


In [20]:

for index, row in df.iterrows():
    tweet_text = row['statement_x']
    
    # Shuffle user prompt tasks to avoid systematic order bias
    user_prompt_sections = user_prompt.split('\n\n')
    random.shuffle(user_prompt_sections)
    shuffled_user_prompt = '\n\n'.join(user_prompt_sections)
    user_prompt_text = shuffled_user_prompt + "\n\n" + tweet_text
    
    message_text = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt_text}]
    
    # Initialize a counter for the number of retries
    retries = 0
    max_retries = 50

    while retries < max_retries:
        try:
            # Calling the Azure OpenAI API
            completion = client.chat.completions.create(
                model="GPT4", # model = "deployment_name"
                messages=message_text,
                temperature=0.7,
                max_tokens=800,
                top_p=0.95,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None
            )
            
            # If the call was successful, break out of the loop
            print('Completed call to API')
            break
        except openai.RateLimitError as e:
            # Handle rate limiting issues
            wait_time = get_wait_time_from_error_message(str(e))
            print(f"RateLimitError: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)  # Retry after the specified time

            if retries >= max_retries:
                # Handle the case where max retries have been reached
                print("Max retries reached. Moving to the next item.")
                continue

        except openai.BadRequestError as e:
            # Handle the BadRequestError by printing the error and moving to the next item
            print(f"BadRequestError occurred: {e}. Skipping to the next item.")
            break  # Break out of the while loop and proceed to the next row

    # Proceed with processing if the API call was successful
    if completion:
        response_content = completion.choices[0].message.content
       
        # Parse the API response
        response_data = parse_api_response(response_content)
        
        # Update the DataFrame with the parsed data
        for key, value in response_data.items():
            if key not in df:
                df[key] = pd.NA
            df.at[index, key] = value
        
        # Print the updated DataFrame row to see the changes
        print(df.loc[index])
        print(df)
        print(index)
        
        # Optionally, save the DataFrame periodically or after each update
        df.to_csv('Politifact_labelled_GPT4_without_NV_missing_rows.csv', index=False) 
        df.to_pickle('Ppolitifact_labelled_GPT4_without_NV_missing_rows.pkl')


Completed call to API
Unnamed: 0.1                                                                       200
Unnamed: 0                                                                        2447
verdict                                                                    mostly-true
statement_originator                                                        Rick Perry
statement_x                          Says Texas high school "graduation rates are a...
statement_date                                                               1/29/2013
statement_source                                                                speech
factchecker                                                           W. Gardner Selby
factcheck_date                                                               2/11/2013
factcheck_analysis_link              https://www.politifact.com/factchecks/2013/feb...
Toxicity_GPT3_wNV                                                                    1
Fact_assessment_GPT3_